# CIFAR10 with CNN
code from https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py


This notebook trains a simple convolutional neural network on the CIFAR10 small images dataset. 



In [1]:
# import libraries
from __future__ import print_function
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

#for confusion matrix
import numpy as np
import sklearn
from sklearn.metrics import confusion_matrix


Using TensorFlow backend.


Prepare data

In [2]:
# define constants
batch_size = 128
num_classes = 10
epochs = 20


# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Cast features into correct data type then scale features
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

170500096/170498071 [==============================] - 21s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Define model

In [3]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


Show model structure

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

Compile model and fit


In [5]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])


model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)




Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 15s 299us/step - loss: 1.9890 - acc: 0.2733 - val_loss: 1.7458 - val_acc: 0.3773
Epoch 2/20
20352/50000 [===========>..................] - ETA: 7s - loss: 1.7569 - acc: 0.3632

50000/50000 [==============================] - 13s 254us/step - loss: 1.7134 - acc: 0.3822 - val_loss: 1.6298 - val_acc: 0.4109
Epoch 3/20
50000/50000 [==============================] - 13s 255us/step - loss: 1.5972 - acc: 0.4225 - val_loss: 1.4859 - val_acc: 0.4644
Epoch 4/20
32896/50000 [==================>...........] - ETA: 4s - loss: 1.5245 - acc: 0.4516

50000/50000 [==============================] - 13s 254us/step - loss: 1.5088 - acc: 0.4570 - val_loss: 1.4500 - val_acc: 0.4727
Epoch 5/20
50000/50000 [==============================] - 13s 254us/step - loss: 1.4302 - acc: 0.4870 - val_loss: 1.3356 - val_acc: 0.5233
Epoch 6/20
35712/50000 [====================>.........] - ETA: 3s - loss: 1.3735 - acc: 0.5112

50000/50000 [==============================] - 13s 253us/step - loss: 1.3679 - acc: 0.5132 - val_loss: 1.2703 - val_acc: 0.5476
Epoch 7/20
50000/50000 [==============================] - 13s 251us/step - loss: 1.3145 - acc: 0.5327 - val_loss: 1.2423 - val_acc: 0.5589
Epoch 8/20
36224/50000 [====================>.........] - ETA: 3s - loss: 1.2674 - acc: 0.5497

50000/50000 [==============================] - 13s 253us/step - loss: 1.2645 - acc: 0.5515 - val_loss: 1.1815 - val_acc: 0.5799
Epoch 9/20
50000/50000 [==============================] - 13s 252us/step - loss: 1.2235 - acc: 0.5687 - val_loss: 1.1433 - val_acc: 0.5965
Epoch 10/20
36224/50000 [====================>.........] - ETA: 3s - loss: 1.1860 - acc: 0.5810

...

50000/50000 [==============================] - 13s 253us/step - loss: 1.1059 - acc: 0.6103 - val_loss: 1.0586 - val_acc: 0.6285
Epoch 13/20
50000/50000 [==============================] - 13s 255us/step - loss: 1.0775 - acc: 0.6211 - val_loss: 0.9928 - val_acc: 0.6549
Epoch 14/20
36224/50000 [====================>.........] - ETA: 3s - loss: 1.0466 - acc: 0.6333

50000/50000 [==============================] - 13s 254us/step - loss: 1.0467 - acc: 0.6323 - val_loss: 0.9848 - val_acc: 0.6521
Epoch 15/20
50000/50000 [==============================] - 13s 254us/step - loss: 1.0175 - acc: 0.6450 - val_loss: 0.9890 - val_acc: 0.6575
Epoch 16/20
36224/50000 [====================>.........] - ETA: 3s - loss: 0.9988 - acc: 0.6508

50000/50000 [==============================] - 13s 254us/step - loss: 0.9960 - acc: 0.6523 - val_loss: 0.9409 - val_acc: 0.6770
Epoch 17/20
50000/50000 [==============================] - 13s 253us/step - loss: 0.9719 - acc: 0.6592 - val_loss: 0.9596 - val_acc: 0.6677
Epoch 18/20
36224/50000 [====================>.........] - ETA: 3s - loss: 0.9559 - acc: 0.6655

50000/50000 [==============================] - 13s 255us/step - loss: 0.9530 - acc: 0.6654 - val_loss: 0.9257 - val_acc: 0.6759
Epoch 19/20
50000/50000 [==============================] - 13s 255us/step - loss: 0.9354 - acc: 0.6736 - val_loss: 0.8829 - val_acc: 0.6941
Epoch 20/20
36224/50000 [====================>.........] - ETA: 3s - loss: 0.9161 - acc: 0.6794

50000/50000 [==============================] - 13s 254us/step - loss: 0.9138 - acc: 0.6806 - val_loss: 0.8781 - val_acc: 0.6973


In [6]:
# Save the original model
model.save('cifra10_base.h5')

In [7]:
# Print base model loss and accuracy
print('\n  - Base case:')
model = keras.models.load_model("cifra10_base.h5")
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])
Y_pred = model.predict(x_test, verbose=2)
cm = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(Y_pred, axis=1))
print('\tConfusion Matrix:\t')
print(cm)


  - Base case:
	Test loss:	 0.8781241746902466
	Test accuracy:	 0.6973
	Confusion Matrix:	
[[733  29  39  18  24   6  10  12  56  73]
 [  4 841   2   4   4   1   8   4  10 122]
 [ 72   8 470  61 165  65  69  55  14  21]
 [ 19  13  51 468 121 147  68  60  14  39]
 [ 21   5  29  44 723  14  47  97  14   6]
 [  9  11  35 179  77 564  27  77   6  15]
 [  8   4  25  54  71  17 784  13   5  19]
 [ 10   5  18  23  85  46   5 781   2  25]
 [ 65  66  11  13   6   5   5   5 769  55]
 [ 16  77   4  10   8   3   6  18  18 840]]


### 1) Augment the data by adding noise.

In [19]:
# Save models with gaussian noise with different Standard Deviation
for e in [0.5,0.1,0.05,0.02,0.01]:  
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  #Add noise
  model.add(keras.layers.GaussianNoise(e))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(x_test, y_test),
                shuffle=True)
  model.save('cifra10_n'+str(e)+'.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 15s 298us/step - loss: 2.2585 - acc: 0.1440 - val_loss: 2.3808 - val_acc: 0.0999
Epoch 2/20
50000/50000 [==============================] - 13s 262us/step - loss: 2.0537 - acc: 0.2404 - val_loss: 2.2931 - val_acc: 0.1495
Epoch 3/20
 8832/50000 [====>.........................] - ETA: 10s - loss: 1.9837 - acc: 0.2656

50000/50000 [==============================] - 13s 262us/step - loss: 1.9297 - acc: 0.2915 - val_loss: 2.1901 - val_acc: 0.1795
Epoch 4/20
50000/50000 [==============================] - 13s 263us/step - loss: 1.7963 - acc: 0.3441 - val_loss: 1.9360 - val_acc: 0.2904
Epoch 5/20
29312/50000 [================>.............] - ETA: 5s - loss: 1.7338 - acc: 0.3667

50000/50000 [==============================] - 13s 263us/step - loss: 1.7223 - acc: 0.3717 - val_loss: 1.8513 - val_acc: 0.3286
Epoch 6/20
50000/50000 [==============================] - 13s 261us/step - loss: 1.6664 - acc: 0.3942 - val_loss: 1.8158 - val_acc: 0.3490
Epoch 7/20
34432/50000 [===================>..........] - ETA: 3s - loss: 1.6228 - acc: 0.4115

50000/50000 [==============================] - 13s 260us/step - loss: 1.6195 - acc: 0.4130 - val_loss: 1.7803 - val_acc: 0.3685
Epoch 8/20
50000/50000 [==============================] - 13s 259us/step - loss: 1.5787 - acc: 0.4292 - val_loss: 1.7900 - val_acc: 0.3656
Epoch 9/20
35712/50000 [====================>.........] - ETA: 3s - loss: 1.5473 - acc: 0.4395

50000/50000 [==============================] - 13s 262us/step - loss: 1.5413 - acc: 0.4422 - val_loss: 1.6679 - val_acc: 0.3968
Epoch 10/20
50000/50000 [==============================] - 13s 263us/step - loss: 1.5066 - acc: 0.4574 - val_loss: 1.5884 - val_acc: 0.4230
Epoch 11/20
35456/50000 [====================>.........] - ETA: 3s - loss: 1.4842 - acc: 0.4645

...

50000/50000 [==============================] - 13s 263us/step - loss: 1.1891 - acc: 0.5782 - val_loss: 1.1177 - val_acc: 0.6085
Epoch 12/20
50000/50000 [==============================] - 13s 263us/step - loss: 1.1571 - acc: 0.5907 - val_loss: 1.0744 - val_acc: 0.6252
Epoch 13/20
35968/50000 [====================>.........] - ETA: 3s - loss: 1.1353 - acc: 0.5994

50000/50000 [==============================] - 13s 261us/step - loss: 1.1288 - acc: 0.6009 - val_loss: 1.0480 - val_acc: 0.6304
Epoch 14/20
50000/50000 [==============================] - 13s 263us/step - loss: 1.1014 - acc: 0.6141 - val_loss: 1.0253 - val_acc: 0.6382
Epoch 15/20
35968/50000 [====================>.........] - ETA: 3s - loss: 1.0785 - acc: 0.6233

50000/50000 [==============================] - 13s 262us/step - loss: 1.0755 - acc: 0.6225 - val_loss: 1.0192 - val_acc: 0.6450
Epoch 16/20
50000/50000 [==============================] - 13s 261us/step - loss: 1.0526 - acc: 0.6302 - val_loss: 1.0146 - val_acc: 0.6428
Epoch 17/20
35456/50000 [====================>.........] - ETA: 3s - loss: 1.0237 - acc: 0.6387

50000/50000 [==============================] - 13s 264us/step - loss: 1.0266 - acc: 0.6377 - val_loss: 0.9683 - val_acc: 0.6630
Epoch 18/20
50000/50000 [==============================] - 13s 263us/step - loss: 1.0062 - acc: 0.6467 - val_loss: 0.9238 - val_acc: 0.6776
Epoch 19/20
35712/50000 [====================>.........] - ETA: 3s - loss: 0.9910 - acc: 0.6507

50000/50000 [==============================] - 13s 263us/step - loss: 0.9849 - acc: 0.6533 - val_loss: 0.9142 - val_acc: 0.6823
Epoch 20/20
50000/50000 [==============================] - 13s 263us/step - loss: 0.9688 - acc: 0.6583 - val_loss: 0.9086 - val_acc: 0.6785


In [20]:
# Load models and print results
for e in [0.5,0.1,0.05,0.02,0.01]:
    model = keras.models.load_model('cifra10_n'+str(e)+'.h5')
    print('\n  - Gaussian Noise with '+str(e)+' stdev:')
    scores = model.evaluate(x_test, y_test, verbose=0)
    print('\tTest loss:\t', scores[0])
    print('\tTest accuracy:\t', scores[1])


  - Gaussian Noise with 0.5 stdev:
	Test loss:	 1.3755655250549317
	Test accuracy:	 0.5045

  - Gaussian Noise with 0.1 stdev:
	Test loss:	 0.9894431276321412
	Test accuracy:	 0.6573

  - Gaussian Noise with 0.05 stdev:
	Test loss:	 0.9525003180503845
	Test accuracy:	 0.6709

  - Gaussian Noise with 0.02 stdev:
	Test loss:	 0.9068649646759033
	Test accuracy:	 0.6836

  - Gaussian Noise with 0.01 stdev:
	Test loss:	 0.9086172308921814
	Test accuracy:	 0.6785


### 2) Add layers to the network to improve performance

#### First model: batch normalization

In [9]:
from keras.layers.normalization import BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)

model.save('cifra10_batch.h5')


Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 18s 366us/step - loss: 1.9586 - acc: 0.3018 - val_loss: 1.5445 - val_acc: 0.4519
Epoch 2/20
50000/50000 [==============================] - 17s 346us/step - loss: 1.5717 - acc: 0.4245 - val_loss: 1.3776 - val_acc: 0.4983
Epoch 3/20
 8576/50000 [====>.........................] - ETA: 13s - loss: 1.4671 - acc: 0.4692

50000/50000 [==============================] - 17s 347us/step - loss: 1.4354 - acc: 0.4785 - val_loss: 1.2801 - val_acc: 0.5419
Epoch 4/20
50000/50000 [==============================] - 17s 347us/step - loss: 1.3556 - acc: 0.5091 - val_loss: 1.2319 - val_acc: 0.5636
Epoch 5/20
29312/50000 [================>.............] - ETA: 6s - loss: 1.3001 - acc: 0.5316

50000/50000 [==============================] - 17s 346us/step - loss: 1.2909 - acc: 0.5355 - val_loss: 1.1479 - val_acc: 0.5922
Epoch 6/20
50000/50000 [==============================] - 18s 350us/step - loss: 1.2318 - acc: 0.5585 - val_loss: 1.1558 - val_acc: 0.5878
Epoch 7/20
34176/50000 [===================>..........] - ETA: 5s - loss: 1.1901 - acc: 0.5721

50000/50000 [==============================] - 17s 348us/step - loss: 1.1776 - acc: 0.5772 - val_loss: 1.0543 - val_acc: 0.6280
Epoch 8/20
50000/50000 [==============================] - 17s 349us/step - loss: 1.1312 - acc: 0.5970 - val_loss: 1.1196 - val_acc: 0.6025
Epoch 9/20
35328/50000 [====================>.........] - ETA: 4s - loss: 1.0907 - acc: 0.6129

50000/50000 [==============================] - 17s 347us/step - loss: 1.0855 - acc: 0.6142 - val_loss: 1.0090 - val_acc: 0.6449
Epoch 10/20
50000/50000 [==============================] - 17s 346us/step - loss: 1.0391 - acc: 0.6311 - val_loss: 0.9533 - val_acc: 0.6633
Epoch 11/20
35200/50000 [====================>.........] - ETA: 4s - loss: 1.0171 - acc: 0.6399

50000/50000 [==============================] - 17s 349us/step - loss: 1.0084 - acc: 0.6418 - val_loss: 0.9583 - val_acc: 0.6641
Epoch 12/20
50000/50000 [==============================] - 18s 350us/step - loss: 0.9709 - acc: 0.6527 - val_loss: 0.9303 - val_acc: 0.6743
Epoch 13/20
35328/50000 [====================>.........] - ETA: 4s - loss: 0.9443 - acc: 0.6664

50000/50000 [==============================] - 17s 348us/step - loss: 0.9379 - acc: 0.6685 - val_loss: 0.8368 - val_acc: 0.7038
Epoch 14/20
50000/50000 [==============================] - 17s 348us/step - loss: 0.9104 - acc: 0.6773 - val_loss: 0.8408 - val_acc: 0.7045
Epoch 15/20
35328/50000 [====================>.........] - ETA: 4s - loss: 0.8859 - acc: 0.6864

50000/50000 [==============================] - 17s 348us/step - loss: 0.8841 - acc: 0.6877 - val_loss: 0.8950 - val_acc: 0.6890
Epoch 16/20
50000/50000 [==============================] - 17s 349us/step - loss: 0.8611 - acc: 0.6959 - val_loss: 0.8247 - val_acc: 0.7083
Epoch 17/20
34944/50000 [===================>..........] - ETA: 4s - loss: 0.8429 - acc: 0.7023

50000/50000 [==============================] - 18s 350us/step - loss: 0.8404 - acc: 0.7033 - val_loss: 0.8125 - val_acc: 0.7171
Epoch 18/20
50000/50000 [==============================] - 18s 353us/step - loss: 0.8185 - acc: 0.7116 - val_loss: 0.7633 - val_acc: 0.7341
Epoch 19/20
34816/50000 [===================>..........] - ETA: 4s - loss: 0.8042 - acc: 0.7191

50000/50000 [==============================] - 17s 347us/step - loss: 0.8051 - acc: 0.7177 - val_loss: 0.7561 - val_acc: 0.7373
Epoch 20/20
50000/50000 [==============================] - 17s 346us/step - loss: 0.7813 - acc: 0.7225 - val_loss: 0.7757 - val_acc: 0.7313


In [10]:

model = keras.models.load_model('cifra10_batch.h5')
print('\n  - Model with Batch Normalization:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])

Y_pred = model.predict(x_test, verbose=2)
cm = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(Y_pred, axis=1))
print('\tConfusion Matrix:\t')
print(cm)


  - Model with Batch Normalization:
	Test loss:	 0.7756728023529053
	Test accuracy:	 0.7313
	Confusion Matrix:	
[[740  11  72  13  44   6  18  24  51  21]
 [ 20 831   8  15   6   5  24   4  25  62]
 [ 46   1 580  34 151  52 104  21   7   4]
 [  8   3  57 451 111 181 150  29   8   2]
 [  7   2  28  25 817  12  75  30   4   0]
 [  5   0  48 147  96 625  42  33   3   1]
 [  3   0  26  30  38   5 893   1   4   0]
 [  8   1  37  18 104  49  16 766   0   1]
 [ 54  25  16  13  13   5  10   2 847  15]
 [ 37  59  10  21  21  12  19  23  35 763]]


#### Second model: regularizer

In [11]:
from keras.layers.normalization import BatchNormalization
from keras import regularizers

weight_decay = 1e-4

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))

model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)

model.save('cifra10_regL2.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 14s 275us/step - loss: 2.0026 - acc: 0.2676 - val_loss: 1.7278 - val_acc: 0.3963
Epoch 2/20
50000/50000 [==============================] - 13s 259us/step - loss: 1.6662 - acc: 0.3998 - val_loss: 1.5401 - val_acc: 0.4575
Epoch 3/20
 9088/50000 [====>.........................] - ETA: 10s - loss: 1.5727 - acc: 0.4316

50000/50000 [==============================] - 13s 259us/step - loss: 1.5268 - acc: 0.4512 - val_loss: 1.4088 - val_acc: 0.5077
Epoch 4/20
50000/50000 [==============================] - 13s 259us/step - loss: 1.4438 - acc: 0.4832 - val_loss: 1.3500 - val_acc: 0.5263
Epoch 5/20
30336/50000 [=================>............] - ETA: 4s - loss: 1.3816 - acc: 0.5116

50000/50000 [==============================] - 13s 257us/step - loss: 1.3747 - acc: 0.5148 - val_loss: 1.3097 - val_acc: 0.5400
Epoch 6/20
50000/50000 [==============================] - 13s 258us/step - loss: 1.3191 - acc: 0.5339 - val_loss: 1.2477 - val_acc: 0.5655
Epoch 7/20
35200/50000 [====================>.........] - ETA: 3s - loss: 1.2760 - acc: 0.5480

50000/50000 [==============================] - 13s 258us/step - loss: 1.2718 - acc: 0.5510 - val_loss: 1.2217 - val_acc: 0.5723
Epoch 8/20
50000/50000 [==============================] - 13s 258us/step - loss: 1.2327 - acc: 0.5660 - val_loss: 1.1381 - val_acc: 0.6027
Epoch 9/20
36224/50000 [====================>.........] - ETA: 3s - loss: 1.1917 - acc: 0.5830

50000/50000 [==============================] - 13s 258us/step - loss: 1.1918 - acc: 0.5829 - val_loss: 1.1052 - val_acc: 0.6133
Epoch 10/20
50000/50000 [==============================] - 13s 258us/step - loss: 1.1593 - acc: 0.5966 - val_loss: 1.0729 - val_acc: 0.6277
Epoch 11/20
35968/50000 [====================>.........] - ETA: 3s - loss: 1.1264 - acc: 0.6065

50000/50000 [==============================] - 13s 259us/step - loss: 1.1288 - acc: 0.6060 - val_loss: 1.0311 - val_acc: 0.6444
Epoch 12/20
50000/50000 [==============================] - 13s 258us/step - loss: 1.1016 - acc: 0.6201 - val_loss: 1.0441 - val_acc: 0.6390
Epoch 13/20
35712/50000 [====================>.........] - ETA: 3s - loss: 1.0792 - acc: 0.6260

50000/50000 [==============================] - 13s 259us/step - loss: 1.0747 - acc: 0.6284 - val_loss: 1.0279 - val_acc: 0.6469
Epoch 14/20
50000/50000 [==============================] - 13s 258us/step - loss: 1.0455 - acc: 0.6400 - val_loss: 0.9752 - val_acc: 0.6595
Epoch 15/20
35968/50000 [====================>.........] - ETA: 3s - loss: 1.0287 - acc: 0.6452

50000/50000 [==============================] - 13s 258us/step - loss: 1.0280 - acc: 0.6440 - val_loss: 0.9799 - val_acc: 0.6637
Epoch 16/20
50000/50000 [==============================] - 13s 260us/step - loss: 1.0059 - acc: 0.6528 - val_loss: 0.9497 - val_acc: 0.6720
Epoch 17/20
35968/50000 [====================>.........] - ETA: 3s - loss: 0.9830 - acc: 0.6628

50000/50000 [==============================] - 13s 258us/step - loss: 0.9849 - acc: 0.6614 - val_loss: 0.9357 - val_acc: 0.6794
Epoch 18/20
50000/50000 [==============================] - 13s 257us/step - loss: 0.9631 - acc: 0.6662 - val_loss: 0.9027 - val_acc: 0.6903
Epoch 19/20
35968/50000 [====================>.........] - ETA: 3s - loss: 0.9423 - acc: 0.6759

50000/50000 [==============================] - 13s 257us/step - loss: 0.9482 - acc: 0.6738 - val_loss: 0.9219 - val_acc: 0.6837
Epoch 20/20
50000/50000 [==============================] - 13s 257us/step - loss: 0.9312 - acc: 0.6795 - val_loss: 0.8839 - val_acc: 0.7000


In [12]:
model= keras.models.load_model('cifra10_regL2.h5')
print('\n  - regularizer L2:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])

Y_pred = model.predict(x_test, verbose=2)
cm = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(Y_pred, axis=1))
print('\tConfusion Matrix:\t')
print(cm)


  - regularizer L2:
	Test loss:	 0.8838677503585816
	Test accuracy:	 0.7
	Confusion Matrix:	
[[714  26  41  25  17   5  15   9 103  45]
 [ 15 858   5  12   4   3  15   3  34  51]
 [ 65   8 535  72 112  67  90  25  14  12]
 [ 15   9  63 525  85 134 113  21  22  13]
 [ 22   4  52  53 656  21 108  61  17   6]
 [  5   4  57 201  57 552  61  47   8   8]
 [  4   3  33  43  33  10 855   5   8   6]
 [ 13   5  28  47  83  60  14 731   4  15]
 [ 48  33  10  15   6   8   9   3 845  23]
 [ 24 123   6  20   8   4  26  15  45 729]]


### 3) Try another method for dealing with overfitting

In [13]:
from keras.layers.normalization import BatchNormalization
from keras import regularizers

weight_decay = 1e-4

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(weight_decay)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)

model.save('cifra10_batch_regL2.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 19s 382us/step - loss: 1.9461 - acc: 0.3075 - val_loss: 1.5003 - val_acc: 0.4614
Epoch 2/20
50000/50000 [==============================] - 18s 353us/step - loss: 1.5682 - acc: 0.4320 - val_loss: 1.3441 - val_acc: 0.5149
Epoch 3/20
 8576/50000 [====>.........................] - ETA: 13s - loss: 1.4778 - acc: 0.4641

50000/50000 [==============================] - 18s 352us/step - loss: 1.4408 - acc: 0.4829 - val_loss: 1.2821 - val_acc: 0.5479
Epoch 4/20
50000/50000 [==============================] - 18s 351us/step - loss: 1.3558 - acc: 0.5142 - val_loss: 1.2352 - val_acc: 0.5592
Epoch 5/20
29568/50000 [================>.............] - ETA: 6s - loss: 1.3038 - acc: 0.5343

50000/50000 [==============================] - 17s 349us/step - loss: 1.2937 - acc: 0.5404 - val_loss: 1.1946 - val_acc: 0.5763
Epoch 6/20
50000/50000 [==============================] - 17s 349us/step - loss: 1.2304 - acc: 0.5645 - val_loss: 1.1992 - val_acc: 0.5793
Epoch 7/20
34176/50000 [===================>..........] - ETA: 5s - loss: 1.1837 - acc: 0.5805

50000/50000 [==============================] - 18s 353us/step - loss: 1.1752 - acc: 0.5842 - val_loss: 1.2086 - val_acc: 0.5912
Epoch 8/20
50000/50000 [==============================] - 18s 353us/step - loss: 1.1254 - acc: 0.6012 - val_loss: 1.0499 - val_acc: 0.6297
Epoch 9/20
35200/50000 [====================>.........] - ETA: 4s - loss: 1.0827 - acc: 0.6187

50000/50000 [==============================] - 18s 353us/step - loss: 1.0783 - acc: 0.6195 - val_loss: 1.0191 - val_acc: 0.6432
Epoch 10/20
50000/50000 [==============================] - 18s 351us/step - loss: 1.0366 - acc: 0.6337 - val_loss: 0.9444 - val_acc: 0.6732
Epoch 11/20
35456/50000 [====================>.........] - ETA: 4s - loss: 1.0022 - acc: 0.6470

50000/50000 [==============================] - 17s 349us/step - loss: 0.9998 - acc: 0.6481 - val_loss: 1.0510 - val_acc: 0.6407
Epoch 12/20
50000/50000 [==============================] - 18s 350us/step - loss: 0.9666 - acc: 0.6589 - val_loss: 0.9194 - val_acc: 0.6855
Epoch 13/20
35200/50000 [====================>.........] - ETA: 4s - loss: 0.9411 - acc: 0.6696

50000/50000 [==============================] - 18s 352us/step - loss: 0.9360 - acc: 0.6712 - val_loss: 0.8710 - val_acc: 0.6987
Epoch 14/20
50000/50000 [==============================] - 18s 352us/step - loss: 0.9062 - acc: 0.6800 - val_loss: 0.8972 - val_acc: 0.6928
Epoch 15/20
35328/50000 [====================>.........] - ETA: 4s - loss: 0.8896 - acc: 0.6912

50000/50000 [==============================] - 18s 350us/step - loss: 0.8848 - acc: 0.6910 - val_loss: 0.8401 - val_acc: 0.7159
Epoch 16/20
50000/50000 [==============================] - 17s 350us/step - loss: 0.8595 - acc: 0.6967 - val_loss: 0.7957 - val_acc: 0.7279
Epoch 17/20
35328/50000 [====================>.........] - ETA: 4s - loss: 0.8423 - acc: 0.7068

50000/50000 [==============================] - 18s 350us/step - loss: 0.8408 - acc: 0.7069 - val_loss: 0.8260 - val_acc: 0.7163
Epoch 18/20
50000/50000 [==============================] - 18s 352us/step - loss: 0.8191 - acc: 0.7123 - val_loss: 0.8365 - val_acc: 0.7121
Epoch 19/20
35712/50000 [====================>.........] - ETA: 4s - loss: 0.8014 - acc: 0.7203

50000/50000 [==============================] - 18s 351us/step - loss: 0.8008 - acc: 0.7206 - val_loss: 0.7795 - val_acc: 0.7325
Epoch 20/20
50000/50000 [==============================] - 18s 351us/step - loss: 0.7840 - acc: 0.7275 - val_loss: 0.8523 - val_acc: 0.7121


In [14]:
model.save('cifra10_batch_regL2.h5')
print('\n  - Model with batch normalization and regularizer:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])


  - Model with batch normalization and regularizer:
	Test loss:	 0.8522795192718506
	Test accuracy:	 0.7121


In [15]:
model = keras.models.load_model('cifra10_batch.h5')
model.add(keras.layers.GaussianNoise(0.01))
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)
model.save('cifra10_batch_n0.01.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 19s 375us/step - loss: 0.8210 - acc: 0.7247 - val_loss: 0.7723 - val_acc: 0.7325
Epoch 2/20
50000/50000 [==============================] - 18s 352us/step - loss: 0.8101 - acc: 0.7312 - val_loss: 0.7703 - val_acc: 0.7334
Epoch 3/20
 8576/50000 [====>.........................] - ETA: 13s - loss: 0.8019 - acc: 0.7367

50000/50000 [==============================] - 18s 354us/step - loss: 0.7963 - acc: 0.7351 - val_loss: 0.7874 - val_acc: 0.7294
Epoch 4/20
50000/50000 [==============================] - 18s 351us/step - loss: 0.7837 - acc: 0.7387 - val_loss: 0.7038 - val_acc: 0.7566
Epoch 5/20
29568/50000 [================>.............] - ETA: 6s - loss: 0.7724 - acc: 0.7468

50000/50000 [==============================] - 18s 351us/step - loss: 0.7750 - acc: 0.7446 - val_loss: 0.7488 - val_acc: 0.7422
Epoch 6/20
50000/50000 [==============================] - 17s 348us/step - loss: 0.7707 - acc: 0.7467 - val_loss: 0.7184 - val_acc: 0.7510
Epoch 7/20
34688/50000 [===================>..........] - ETA: 5s - loss: 0.7457 - acc: 0.7525

50000/50000 [==============================] - 17s 350us/step - loss: 0.7496 - acc: 0.7520 - val_loss: 0.8515 - val_acc: 0.7062
Epoch 8/20
50000/50000 [==============================] - 18s 353us/step - loss: 0.7456 - acc: 0.7530 - val_loss: 0.7395 - val_acc: 0.7487
Epoch 9/20
35328/50000 [====================>.........] - ETA: 4s - loss: 0.7339 - acc: 0.7557

50000/50000 [==============================] - 18s 352us/step - loss: 0.7329 - acc: 0.7565 - val_loss: 0.7490 - val_acc: 0.7435
Epoch 10/20
50000/50000 [==============================] - 18s 352us/step - loss: 0.7259 - acc: 0.7621 - val_loss: 0.6818 - val_acc: 0.7642
Epoch 11/20
34944/50000 [===================>..........] - ETA: 4s - loss: 0.7129 - acc: 0.7618

50000/50000 [==============================] - 17s 349us/step - loss: 0.7176 - acc: 0.7622 - val_loss: 0.6787 - val_acc: 0.7663
Epoch 12/20
50000/50000 [==============================] - 18s 352us/step - loss: 0.7034 - acc: 0.7679 - val_loss: 0.7400 - val_acc: 0.7453
Epoch 13/20
35328/50000 [====================>.........] - ETA: 4s - loss: 0.6931 - acc: 0.7717

50000/50000 [==============================] - 18s 351us/step - loss: 0.7027 - acc: 0.7711 - val_loss: 0.7097 - val_acc: 0.7578
Epoch 14/20
50000/50000 [==============================] - 18s 354us/step - loss: 0.6790 - acc: 0.7745 - val_loss: 0.6537 - val_acc: 0.7727
Epoch 15/20
35072/50000 [====================>.........] - ETA: 4s - loss: 0.6829 - acc: 0.7755

50000/50000 [==============================] - 18s 353us/step - loss: 0.6843 - acc: 0.7749 - val_loss: 0.7396 - val_acc: 0.7472
Epoch 16/20
50000/50000 [==============================] - 17s 349us/step - loss: 0.6828 - acc: 0.7742 - val_loss: 0.6643 - val_acc: 0.7729
Epoch 17/20
34816/50000 [===================>..........] - ETA: 4s - loss: 0.6569 - acc: 0.7845

50000/50000 [==============================] - 18s 350us/step - loss: 0.6659 - acc: 0.7816 - val_loss: 0.6720 - val_acc: 0.7682
Epoch 18/20
50000/50000 [==============================] - 17s 349us/step - loss: 0.6553 - acc: 0.7839 - val_loss: 0.6703 - val_acc: 0.7711
Epoch 19/20
35072/50000 [====================>.........] - ETA: 4s - loss: 0.6517 - acc: 0.7834

50000/50000 [==============================] - 18s 352us/step - loss: 0.6529 - acc: 0.7834 - val_loss: 0.6836 - val_acc: 0.7681
Epoch 20/20
50000/50000 [==============================] - 18s 351us/step - loss: 0.6535 - acc: 0.7891 - val_loss: 0.6446 - val_acc: 0.7755


In [16]:
model = keras.models.load_model('cifra10_batch_n0.01.h5')

print('\n  - Model with Batch Normalization and Gaussian Noise with 0.01 stdev:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])


  - Model with Batch Normalization and Gaussian Noise with 0.01 stdev:
	Test loss:	 0.6446445227622986
	Test accuracy:	 0.7755
